In [ ]:
!pip install cirq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.6/45.6 kB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 27.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 532.7/532.7 kB 37.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.5/60.5 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.3/69.3 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 596.5/596.5 kB 37.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 202.6/202.6 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 75.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 74.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 64.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.8/117.8 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━

# Encriptación

In [ ]:
import math
import random
from fractions import Fraction
import cirq
import numpy as np

def generate_prime(bits):
    while True:
        n = random.getrandbits(bits)
        if n > 2 and all(n % i != 0 for i in range(2, int(math.sqrt(n)) + 1)):
            return n

def generate_keypair(bits=8):
    p = generate_prime(bits)
    q = generate_prime(bits)
    n = p * q
    phi = (p-1) * (q-1)
    e = 65537
    d = pow(e, -1, phi)
    return ((e, n), (d, n))

def encrypt(pk, plaintext):
    e, n = pk
    return pow(plaintext, e, n)

# Ejemplo de uso
public, private = generate_keypair(8)  # Usa 8 bits para generar los números primos

# Solicitar el mensaje al usuario
max_message = min(2**15 - 1, public[1] - 1)
message = int(input(f"Ingrese un número entre 0 y {max_message} para encriptar: "))
if message < 0 or message > max_message:
    raise ValueError(f"El mensaje debe estar entre 0 y {max_message}")

# Encriptar
encrypted = encrypt(public, message)
print(f"Mensaje encriptado: {encrypted}")

# Guardar las claves y el mensaje encriptado
with open('public_key.txt', 'w') as f:
    f.write(f"{public[0]},{public[1]}")
with open('private_key.txt', 'w') as f:
    f.write(f"{private[0]},{private[1]}")
with open('encrypted_message.txt', 'w') as f:
    f.write(str(encrypted))


Ingrese un número entre 0 y 1762 para encriptar: 1234
Mensaje encriptado: 148


# Desencriptación

In [ ]:
import math
import random
from fractions import Fraction
import cirq
import numpy as np

def decrypt(pk, ciphertext):
    d, n = pk
    return pow(ciphertext, d, n)

def quantum_phase_estimation(a, N):
    n = N.bit_length()
    qubits = cirq.LineQubit.range(2*n)
    circuit = cirq.Circuit()

    # Inicializar el estado |1>
    circuit.append(cirq.X(qubits[n]))

    # Aplicar H a los primeros n qubits
    circuit.append(cirq.H.on_each(qubits[:n]))

    # Aplicar operaciones controladas
    for i in range(n):
        circuit.append(cirq.CZPowGate(exponent=a*2**i/N).on(qubits[i], qubits[n]))

    # Aplicar QFT inversa
    circuit.append(cirq.inverse(cirq.qft(*qubits[:n])))

    # Medir
    circuit.append(cirq.measure(*qubits[:n], key='result'))

    return circuit

def shor_algorithm_simulation(N):
    a = random.randint(2, N-1)
    if math.gcd(a, N) != 1:
        return math.gcd(a, N), N // math.gcd(a, N)

    circuit = quantum_phase_estimation(a, N)

    # En lugar de ejecutar el circuito, simularemos el resultado
    # Esto es una simplificación y no representa el verdadero comportamiento cuántico
    phase = random.random()

    # Convertir la fase en una fracción
    frac = Fraction(phase).limit_denominator(N)
    r = frac.denominator

    if r % 2 == 0:
        guesses = [math.gcd(pow(a, r//2, N) - 1, N), math.gcd(pow(a, r//2, N) + 1, N)]
        for guess in guesses:
            if 1 < guess < N:
                return guess, N // guess

    return "Fallo al factorizar"

# Cargar las claves y el mensaje encriptado
with open('public_key.txt', 'r') as f:
    public = tuple(map(int, f.read().split(',')))
with open('private_key.txt', 'r') as f:
    private = tuple(map(int, f.read().split(',')))
with open('encrypted_message.txt', 'r') as f:
    encrypted = int(f.read())

# Usar Shor para factorizar n
n = public[1]
factors = shor_algorithm_simulation(n)
print(f"Factores encontrados: {factors}")

if isinstance(factors, tuple):
    # Reconstruir la clave privada
    p, q = factors
    phi = (p-1) * (q-1)
    d = pow(public[0], -1, phi)
    cracked_private = (d, n)

    # Desencriptar usando la clave privada reconstruida
    decrypted = decrypt(cracked_private, encrypted)
    print(f"Mensaje desencriptado: {decrypted}")

    # Verificar si la desencriptación fue exitosa
    if decrypted == message:
        print("La desencriptación fue exitosa. El mensaje original fue recuperado.")
    else:
        print("La desencriptación falló. El mensaje recuperado no coincide con el original.")
else:
    print("No se pudo factorizar n. Intente de nuevo.")




Factores encontrados: (41, 43)
Mensaje desencriptado: 1234
La desencriptación fue exitosa. El mensaje original fue recuperado.
